# Emerging Technologies

This notebook contains solutions for the assessment problems on classical and quantum algorithms using the Deutsch–Jozsa algorithm.

## Import Required

In [87]:
import random
import numpy as np


## Introduction

### Classical vs Quantum Information

**[Classical Information](https://en.wikipedia.org/wiki/Classical_information_theory)** - Think about how a CD or DVD stores music and videos. The disc surface contains tiny physical pits — a pit represents a 0, and a flat area (called a "land") represents a 1. When you play the disc, a laser reads these pits one by one, [decoding the binary data sequentially.](https://www.sciencedirect.com/topics/engineering/compact-disc) This is how classical computers work: they process information as definite 0s and 1s, checking each value individually. If you wanted to find out whether a mystery function always gives the same answer (constant) or gives a mix of answers (balanced), you would have to test many inputs one at a time — potentially needing up to 9 queries for a 4-input function.

**[Quantum Information](https://en.wikipedia.org/wiki/Quantum_information)** - Now imagine you need to decide whether to bring a raincoat before going outside. Classically, you would check the weather forecast first, then decide. But what if you could somehow be prepared for all weather possibilities at once? This is the essence of [quantum superposition](https://en.wikipedia.org/wiki/Quantum_superposition) — a quantum bit (qubit) can exist as both 0 AND 1 simultaneously, like being in a state of "maybe rain, maybe sunshine" until you observe it. The [Deutsch–Jozsa algorithm](https://arxiv.org/abs/quant-ph/9708016) exploits this property: by putting qubits into superposition, we can query a function with all possible inputs at once, and through quantum interference, the answer (constant or balanced) reveals itself in just a single query. This is the quantum advantage we will explore in this notebook.


## Problem 1: Generating Random Boolean Functions

**Instructions:**  
Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean inputs.

### 1. Classical Systems and Their State Sets

A classical system is defined by the set of states it can be in.

Examples:

- If \(X\) is a **bit**, then  
Σ = \{0,1\}

- If \(X\) is a **six-sided die**, then  
Σ = \{1,2,3,4,5,6\}


- If \(X\) is a **fan switch**, then  
Σ = {high, medium, low, off}

The physical representation does not matter — only the distinct states matter.

“A bit is just a system that can be in two different states.”

### 2. When We Know the State Exactly

Sometimes we know the state with certainty.

Example:  
If the fan switch is set to *high*, we know the exact classical state.

But in real computation (e.g., networking), we often **don’t** know the state.

### 3. When We Don’t Know the State: Probabilistic States

“If you do any network programming, you’re going to take bits from the network, and you don’t know what they are.”

Real-world information is often uncertain. You cannot predict incoming bits. If you could predict them, the channel would carry less information (because information = unpredictability).

Suppose we believe:

The probability of \(X=0\) is $ \Pr(X=0) = \frac{3}{4} $, and the probability of $X=1$ is $ \Pr(X=1) = \frac{1}{4} $.

This is exactly the IBM Quantum textbook example.

### 4. Representing Uncertainty with Probability Vectors

We represent the probabilistic state as a **column vector**:

\[
\begin{bmatrix}
0.75 \\
0.25
\end{bmatrix}
\]

- Top entry = probability of 0  
- Bottom entry = probability of 1  
- They must sum to 1  

A **probability vector** must satisfy:

1. All entries ≥ 0  
2. Entries sum to 1  

This is the classical analogue of quantum state vectors.

### 5. Special Probability Vectors: The Basis States

Two special vectors represent definite classical states:

$$
|0\rangle = \begin{bmatrix}1 \\ 0\end{bmatrix}, \qquad
|1\rangle = \begin{bmatrix}0 \\ 1\end{bmatrix}.
$$

These are the **basis vectors** of classical information. Any probability vector can be written as:

$$
p(0)\,|0\rangle + p(1)\,|1\rangle.
$$

### 6. Demonstrating Probability Vectors in Python




In [88]:
# Example probability vector
p = np.array([[0.75],
              [0.25]])
# Check that the probabilities sum to 1
p.sum()


np.float64(1.0)

### 7. Classical Functions on a Single Bit

A function that takes one bit in and outputs one bit can only behave in **four** possible ways:

| Name | Description | Mapping |
|------|-------------|---------|
| F1 | Constant 0 | 0→0, 1→0 |
| F2 | Identity | 0→0, 1→1 |
| F3 | NOT | 0→1, 1→0 |
| F4 | Constant 1 | 0→1, 1→1 |

“No matter how complicated thecode is, at the end it’s one of these four.”

### 8. Implementing the Four Functions in Python

In [89]:
def f1(a):
    return 0

def f2(a):
    return a

def f3(a):
    return 1 - a

def f4(a):
    return 1

functions = [f1, f2, f3, f4]

### 9. Choosing a Random Function

In [90]:
f = random.choice(functions) 
f

<function __main__.f4(a)>

### 10. Identifying the Random Function

“The only thing can do is ask the function what happens if put in 0 and what happens if put in 1.”

In [91]:
f(0), f(1)

(1, 1)

### 11. Determining Which Function It Is

We compare the outputs to the four possibilities.

In [92]:
def identify_function(f):
    out0 = f(0)
    out1 = f(1)
    
    if out0 == 0 and out1 == 0:
        return "F1 (constant 0)"
    if out0 == 0 and out1 == 1:
        return "F2 (identity)"
    if out0 == 1 and out1 == 0:
        return "F3 (NOT)"
    if out0 == 1 and out1 == 1:
        return "F4 (constant 1)"

identify_function(f)

'F4 (constant 1)'

### 12. Why This Matters for Quantum Computing

Classically, we need **two queries** to fully identify the function.

Quantum computing asks:

> Can we learn something about the function with **one** query?

This leads directly to **Deutsch’s algorithm**, the first example of quantum speedup.

---

### Random Functions (classical toy problem)

One of the simplest toy problems for introducing quantum speedups is to work with raw bits.  
This notebook sets up functions $f:\{0,1\}^n \to \{0,1\}$ that are guaranteed to be either **constant** or **balanced**.  
The $n=3$ example, the random-tuple representation, the combinatorics for balanced functions, and Python code to generate and test random functions are included.

#### Purpose

The problem is reduced to its barest form: functions on bits. No semantic meaning is attached to inputs or outputs — they are raw bits.  
The task: given oracle access to a function $f$ that is either constant or balanced, determine which type it is. The classical baseline requires many queries; this sets up the Deutsch / Deutsch–Jozsa style question.

#### Definitions

- **Constant function:** the output is identical for every input. For a length-$2^n$ tuple the output is either all zeros or all ones.  
- **Balanced function:** exactly half of the $2^n$ outputs are $0$ and half are $1$ (so there are $2^{n-1}$ zeros and $2^{n-1}$ ones).

#### Example: $n = 3$ (table representation)

All input triples are listed and one possible function output column is shown. The function column is a length-$2^n$ tuple.

| **a** | **b** | **c** | **f(a,b,c)** |
|-----:|-----:|-----:|:------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |

- **Constant examples:** $(0,0,0,0,0,0,0,0)$ and $(1,1,1,1,1,1,1,1)$.  
- **Balanced examples:** any tuple with exactly four $0$'s and four $1$'s, e.g. $(0,0,0,0,1,1,1,1)$ or $(1,0,0,1,1,0,0,1)$.


In [93]:
# Problem 1 solution goes here
# Deterministic operations
def random_constant_balanced():
    """
    Returns a randomly chosen constant or balanced function
    that takes 4 Boolean inputs.
    """
    # Step 1: Randomly decide constant or balanced
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Step 2a: Constant function - all 16 outputs are the same
        value = random.choice([0, 1])
        outputs = [value] * 16
    else:
        # Step 2b: Balanced function - 8 zeros and 8 ones, shuffled
        outputs = [0] * 8 + [1] * 8
        random.shuffle(outputs)
    
    # Step 3: Return a function that uses this lookup table
    def f(a, b, c, d):
        # Convert 4 bits to index (0-15)
        index = 8*a + 4*b + 2*c + d
        return outputs[index]
    
    return f

In [94]:
# Test the function
f = random_constant_balanced()

# Print all outputs to see the pattern
print("Input -> Output")
for i in range(16):
    a = (i >> 3) & 1
    b = (i >> 2) & 1
    c = (i >> 1) & 1
    d = i & 1
    print(f"({a},{b},{c},{d}) -> {f(a,b,c,d)}")

# Count outputs to verify constant/balanced
outputs = [f((i>>3)&1, (i>>2)&1, (i>>1)&1, i&1) for i in range(16)]
zeros = outputs.count(0)
ones = outputs.count(1)
print(f"\nZeros: {zeros}, Ones: {ones}")
print(f"Type: {'Constant' if zeros == 0 or zeros == 16 else 'Balanced'}")

Input -> Output
(0,0,0,0) -> 1
(0,0,0,1) -> 1
(0,0,1,0) -> 1
(0,0,1,1) -> 1
(0,1,0,0) -> 1
(0,1,0,1) -> 1
(0,1,1,0) -> 1
(0,1,1,1) -> 1
(1,0,0,0) -> 1
(1,0,0,1) -> 1
(1,0,1,0) -> 1
(1,0,1,1) -> 1
(1,1,0,0) -> 1
(1,1,0,1) -> 1
(1,1,1,0) -> 1
(1,1,1,1) -> 1

Zeros: 0, Ones: 16
Type: Constant


## Problem 2: Classical Testing for Function Type

**Instructions:**  
Write a Python function `determine_constant_balanced(f)` that analyzes a function f and returns "constant" or "balanced". Include a brief note on efficiency.

In [95]:
# Problem 2 solution goes here

## Problem 3: Quantum Oracles

**Instructions:**  
Using Qiskit, create quantum oracles for each of the possible single-Boolean-input functions in Deutsch's algorithm. Explain how each oracle works.

In [96]:
# Problem 3 solution goes here

## Problem 4: Deutsch's Algorithm with Qiskit

**Instructions:**  
Design a quantum circuit that solves Deutsch's problem for single-input functions. Demonstrate its use with each oracle and explain the interference pattern.

In [97]:
# Problem 4 solution goes here

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

**Instructions:**  
Use Qiskit to create a quantum circuit for four-bit functions generated in Problem 1. Demonstrate it on constant and balanced functions and explain the results.

In [98]:
# Problem 5 solution goes here